## Replace Title

##### Assistant

In [56]:
from pathlib import Path
from bs4 import BeautifulSoup as bs
import requests
from fastcore.net import urlsave
from fastcore.utils import Path

In [2]:
def update_title(html_path: str|Path, default_title: str = "nbsanity | Jupyter Notebook Viewer"):
    doc = Path(html_path)
    soup = bs(doc.read_text(encoding='utf-8'), 'html.parser')
    title = soup.title.string if soup.title else default_title
    if meta_tag := soup.find('meta', property='og:title'): meta_tag['content'] = title
    doc.write_text(str(soup), encoding='utf-8')

##### User

In [3]:
# !rm -rf static/307cfee0a3f8f7d76b7646960ea599f0

We need to replace the below placeholder with the title from the head

In [4]:
!cat static/307cfee0a3f8f7d76b7646960ea599f0/long_tail.html | grep "og:title"

<meta property="og:title" content="{{{title}}}">


Let's check if it works!

In [5]:
update_title('static/307cfee0a3f8f7d76b7646960ea599f0/long_tail.html')

In [6]:
!cat static/307cfee0a3f8f7d76b7646960ea599f0/long_tail.html | grep "og:title"

<meta content="Stitch Fix, Jupyter, GitHub, and the Long Tail" property="og:title"/>


## Test shotscraper

In [57]:
def gist_raw(gist_url):
    # Extract gist ID from URL
    gist_id = gist_url.split('/')[-1]
    
    # Get gist info from GitHub API
    api_url = f"https://api.github.com/gists/{gist_id}"
    response = requests.get(api_url)
    data = response.json()
    
    # Check if 'files' or 'raw_url' is missing and raise HTTPError
    if 'files' not in data or not data['files']:
        raise urllib.error.HTTPError(api_url, 404, "No files found in the gist", None, None)
    
    raw_url = list(data['files'].values())[0].get('raw_url')
    if not raw_url:
        raise urllib.error.HTTPError(api_url, 404, "No raw_url found for the file", None, None)
    
    return raw_url


def git2raw(git_url: str): 
    if git_url.startswith('https://gist.github.com/'): return gist_raw(git_url)
    if git_url.startswith('https://gist.githubusercontent.com') and 'raw' in git_url: return git_url
    return git_url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")


def escape_filename(filename):
    # Replace HTML-like characters and other problematic characters
    invalid_chars = '<>:"/\\|?*%#&{}+=`@!^;[]()$'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

Let's test it:

In [76]:
!rm -rf temp/
!mkdir temp

In [77]:

full_url='https://gist.github.com/93degree/f979a600bd6ef75044e83c3d9bfd6022'
nm = urlsave(git2raw(full_url), 'temp')
nm

Path('temp/AoC_2023%2C_day_1.ipynb')

In [80]:
nm.rename(nm.parent/escape_filename(nm.name))

Path('temp/AoC_2023_2C_day_1.ipynb')

In [81]:
!ls temp/

AoC_2023_2C_day_1.ipynb
